## **Import Libraries**

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.stats import norm


## **Set time for certain number of years**



In [ ]:
years =15
endDate= dt.datetime.now()
startDate= endDate-dt.timedelta(days=365*years)

## **Add Tickers**

In [ ]:
tickers=["NTPC.NS","TCS.NS","ITC.NS","CIPLA.NS","RELIANCE.NS"]

## **Downloading stock prices using for loop**

In [ ]:
adj_close_df= pd.DataFrame()

for ticker in tickers:
  data= yf.download(ticker,start=startDate,end=endDate)
  adj_close_df[ticker]= data["Close"]

print(adj_close_df)


## **Calculating LN return and droping NA**

In [ ]:
log_returns= np.log((adj_close_df/adj_close_df.shift(1)))
log_returns= log_returns.dropna()
print(log_returns)

## **Calculate Portfolio Expected Return**

In [ ]:
def expected_return(weights,log_returns):
  return np.sum(log_returns.mean()*weights)

## **Calculate Portfolio Stdev**

In [ ]:
def stdev(weights,cov_matrix):
  variance = weights.T @ cov_matrix @ weights
  return np.sqrt(variance)

## **Calculate Portfolio Cov matrix**

In [ ]:
cov_matrix= log_returns.cov()
print(cov_matrix)

Weights, Returns and Stdev

In [ ]:
portfolio_value = 10000000
weights = np.array([1/len(tickers)]*len(tickers))
portfolio_expected_return= expected_return(weights,log_returns)
portfolio_stdev= stdev(weights,cov_matrix)

## **Generating Random z values**

In [ ]:
def random_z_score():
  return np.random.normal(0,1)

## **Creating gain/loss scenario**

In [ ]:
days = 15
def scenario_gain_loss(portfolio_value,portfolio_stdev,z_score,days):
  return (portfolio_value * portfolio_expected_return * days) + (portfolio_value*portfolio_stdev * z_score * np.sqrt(days))


## **Run 10,000 simulation**



In [ ]:
simulation = 10000
scenario_return = []
for i in range(simulation):
  z_score = random_z_score()
  scenario_return.append(scenario_gain_loss(portfolio_value,portfolio_stdev,z_score,days))

confidence_interval = 0.95
VaR = np.percentile(scenario_return,100*(1-confidence_interval))
print(-VaR)

## **Ploting the results of simulation**

In [ ]:
plt.hist(scenario_return,bins=50, density=True)
plt.xlabel("Scenario Gain/Loss Rs")
plt.ylabel("Frequency")
plt.title(f"Distribution of Scenario Gain/Loss over {days} days")
plt.axvline(VaR, color = "r", linestyle = "dashed", linewidth = 2, label = f"VaR at {confidence_interval:.0%} confidence interval")
plt.legend()
plt.show()